In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.9 MB/s eta 0:00:00


In [ ]:
import re
import json
from sklearn.model_selection import train_test_split

In [10]:
with open('/content/IMDB_reviews.json', encoding= 'utf-8') as f:
  text = f.readlines()
  data = []
  for line in text[0:80001]:
        pyth_data = json.loads(line)
        data.append(pyth_data)
  

In [11]:
data[0]

{'review_date': '10 February 2006',
 'movie_id': 'tt0111161',
 'user_id': 'ur1898687',
 'is_spoiler': True,
 'review_text': 'In its Oscar year, Shawshank Redemption (written and directed by Frank Darabont, after the novella Rita Hayworth and the Shawshank Redemption, by Stephen King) was nominated for seven Academy Awards, and walked away with zero. Best Picture went to Forrest Gump, while Shawshank and Pulp Fiction were "just happy to be nominated." Of course hindsight is 20/20, but while history looks back on Gump as a good film, Pulp and Redemption are remembered as some of the all-time best. Pulp, however, was a success from the word "go," making a huge splash at Cannes and making its writer-director an American master after only two films. For Andy Dufresne and Co., success didn\'t come easy. Fortunately, failure wasn\'t a life sentence.After opening on 33 screens with take of $727,327, the $25M film fell fast from theatres and finished with a mere $28.3M. The reasons for failure 

In [30]:
def add_spoiler_tag (data_list):
    new_data_list = []
    for elem in data_list:
        origin_review_text = ' '.join(elem['review_text'].split(' ')[0:101])
        if elem['is_spoiler'] == True:
            elem['review_text'] = origin_review_text + ' is_spoiler' + ' true'
            new_data_list.append(elem)
        else:
            elem['review_text'] = origin_review_text + ' is_spoiler' + ' false'
            new_data_list.append(elem)
    return new_data_list

In [31]:
train, test = train_test_split(data,test_size=0.2) 
print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 64000
Test dataset length: 16001


In [37]:
def build_text_files(data_list, dest_path):
  with open (dest_path, 'w', encoding = 'utf-8')as f:
    data_l = []
    for wlwm in data_list:
      data_l.append(wlwm ['review_text'])
    f.write(' '.join(data_l))
    

В трейне добавляем специальный тег ("spoiler") к каждому тексту review, который будет сигнализировать, что дальше идет определение наличия/отсутствия спойлера в этом тексте

In [38]:
train = add_spoiler_tag(train)
train[0:3]

[{'review_date': '5 April 2005',
  'movie_id': 'tt0116282',
  'user_id': 'ur4798799',
  'is_spoiler': True,
  'review_text': "Fargo is one of my favorite films of all time and in my opinion deserves to be in the top 10 films of the 90's. From the onset of the movie, with views of the snow covered North Dakota landscape and the beautiful and haunting music of composer Carter Burwell playing in the background until the very last scene, Fargo never lets up. This film is Joel and Ethan Cohen's masterpiece, a truly original and one of a kind work. Every aspect of the film is handled with the utmost precision and care. The screenplay, written by Joel and Ethan, is is_spoiler true",
  'rating': '9',
  'review_summary': 'Excellent Film'},
 {'review_date': '15 August 2011',
  'movie_id': 'tt0078748',
  'user_id': 'ur25837689',
  'is_spoiler': False,
  'review_text': "A ship's investigation of a distress signal given off on a distant planet introduces them to a savage creature that begins to hun

In [39]:
build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [40]:
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [22]:
import torch

In [41]:
torch.cuda.empty_cache()


Обучаем модельGPT2

In [19]:
from transformers import GPT2Tokenizer, GPT2Model

In [42]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [43]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [47]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead
model = AutoModelWithLMHead.from_pretrained("gpt2")


/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Обучение модели:

In [48]:
training_args = TrainingArguments(
    output_dir="./gpt2-chef", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500, # number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [49]:
trainer.train()

Step,Training Loss
500,3.701400
1000,3.515100
1500,3.461700
2000,3.428100
2500,3.364700
3000,3.355900
3500,3.350300


Step,Training Loss
500,3.701400
1000,3.515100
1500,3.461700
2000,3.428100
2500,3.364700
3000,3.355900
3500,3.350300


TrainOutput(global_step=3924, training_loss=3.4419599527248184, metrics={'train_runtime': 4229.8568, 'train_samples_per_second': 29.686, 'train_steps_per_second': 0.928, 'total_flos': 8202348822528000.0, 'train_loss': 3.4419599527248184, 'epoch': 2.0})

In [50]:
trainer.save_model()

## Инференс

In [51]:
from transformers import TFGPT2LMHeadModel


In [52]:
model = TFGPT2LMHeadModel.from_pretrained("gpt2-chef", from_pt=True, pad_token_id=tokenizer.eos_token_id)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.1.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [53]:
from transformers import pipeline
chef = pipeline('text-generation',model='gpt2-chef', tokenizer='gpt2')

In [54]:
chef("Fargo is one of my favorite films of all time and in my opinion deserves to be in the top 10 films of the 90's.")

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': "Fargo is one of my favorite films of all time and in my opinion deserves to be in the top 10 films of the 90's. The performances by Leonardo DiCaprio, Samuel L. Jackson, and Michael Caine all play good roles but"}]

In [55]:
input_text1 = "Fargo is one of my favorite films of all time and in my opinion deserves to be in the top 10 films of the 90's."
input_text2 = "Heath Ledger as the Joker really did a great job on the role, no denying that"

In [56]:
import tensorflow as tf


In [63]:
def spoiler_detect(generated_text):
  new = str(generated_text) + " is_spoiler"
  return new


In [91]:
test_sent_list = []
for elem in test:
  sent = elem['review_text'].split('.')[0]
  test_sent_list.append(sent)

In [92]:
len(test_sent_list)

16001

Жадный поиск

In [68]:
tf.random.set_seed(0)
input_ids = tokenizer.encode(input_text1,  return_tensors='tf')
tf.random.set_seed(0)
greedy_output = model.generate(input_ids, max_length=90)
output = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Fargo is one of my favorite films of all time and in my opinion deserves to be in the top 10 films of the 90's. It is a film that is so well made and so well acted that it is hard to believe that it was made in the 90's. The acting is superb and the story is so well told that it is hard to believe that it was made in the 90's. The acting is superb and the story is


In [69]:
input_ids2 = tokenizer.encode(input_text2,  return_tensors='tf')
tf.random.set_seed(0)
greedy_output2 = model.generate(input_ids2, max_length=90)
print(tokenizer.decode(greedy_output2[0], skip_special_tokens=True))

Heath Ledger as the Joker really did a great job on the role, no denying that he was a great actor. The Joker was a very good actor, but I think he was a bit too much of a villain for the role. I think that Heath Ledger was a bit too much of a villain for the role. I think that Heath Ledger was a bit too much of a villain for the role. I think that Heath Led


In [ ]:
greedy_list = []
for sent in test_sent_list[0:501]:
  if len(sent)<=90:
    input_id = tokenizer.encode(sent, return_tensors='tf')
    tf.random.set_seed(6)
    greedy_output = model.generate(input_id, max_length=90)
    greedy_list.append(tokenizer.decode(greedy_output[0], skip_special_tokens=True))
len_list = len(greedy_list)
greedy_spoil_alert_counter = 0
for elem in greedy_list:
  if "<spoiler>" in elem:
    greedy_spoil_alert_counter+=1

result = greedy_spoil_alert_counter/500
print(result)

Beam search

In [70]:
tf.random.set_seed(0)
beam_output = model.generate(
    input_ids,  
    max_length=90, 
    num_beams=5, 
    early_stopping=True
)

print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Fargo is one of my favorite films of all time and in my opinion deserves to be in the top 10 films of the 90's. The acting is top notch, the direction is top notch, the story is top notch, and the acting is top notch. The acting is top notch, the direction is top notch, and the story is top notch. The acting is top notch, the direction is top notch, and the story is top notch


In [71]:
tf.random.set_seed(0)
beam_output2 = model.generate(
    input_ids2,  
    max_length=90, 
    num_beams=5, 
    early_stopping=True
)

print(tokenizer.decode(beam_output2[0], skip_special_tokens=True))

Heath Ledger as the Joker really did a great job on the role, no denying that he did a great job as the Joker, but he also did a great job as the Joker's father. The Joker's father is the Joker's father, and the Joker's father is the Joker's father. The Joker's father is the Joker's father, and the Joker's father is the Joker's father. The Joker's father is the Joker


In [ ]:
beam_list = []
for sent in test_sent_list[0:501]:
  if len(sent)<=90:
      input_id1 = tokenizer.encode(sent, return_tensors='tf')
      tf.random.set_seed(6)
      beam_output = model.generate(input_id1,  max_length=90, num_beams=5, early_stopping=True)
      beam_list.append(tokenizer.decode(beam_output[0], skip_special_tokens=True))
len_list = len(beam_list)
beam_spoil_alert_counter = 0
for elem in beam_list:
  if "<spoiler>" in elem:
    beam_spoil_alert_counter+=1

result_beam = beam_spoil_alert_counter/500
print(result_beam)

Sampling

In [72]:
tf.random.set_seed(0)
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=90, 
    top_k=0
)

print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Fargo is one of my favorite films of all time and in my opinion deserves to be in the top 10 films of the 90's. It is on IMDb 100% so I'm not going to stop trying to comment on it nevertheless.The ending I was pounced on at the end is nothing short of a precursor to Fight Club. It seems to me this is exactly like Brian De Palma's Double Indemnity' ending the <


In [74]:
tf.random.set_seed(0)
sample_output2 = model.generate(
    input_ids2, 
    do_sample=True, 
    max_length=90, 
    top_k=0
)

print(tokenizer.decode(sample_output2[0], skip_special_tokens=True))

Heath Ledger as the Joker really did a great job on the role, no denying that the film was somewhat of a disappointment, although I was pleased to see Heath Ledger deliver his first bizarre action hero role, pitting a suave banker against an unlikely hero. Punished but effervescent as <spoiler> true  People don't like novels, but I just can't find anyone who pays any attention to the vast net of


In [ ]:
samol_list = []
for sent in test_sent_list[0:501]:
  if len(sent)<=90:
      input_id2 = tokenizer.encode(sent, return_tensors='tf')
      tf.random.set_seed(6)
      sampl_output = model.generate(input_id2, do_sample=True, max_length=90, top_k=0)
      samol_list.append(tokenizer.decode(sampl_output[0], skip_special_tokens=True))
len_list = len(samol_list)
sampl_spoil_alert_counter = 0
for elem in samol_list:
  if "<spoiler>" in elem:
    sampl_spoil_alert_counter+=1

result_sampl = sampl_spoil_alert_counter/500
print(result_sampl)

Top-k sampling

In [76]:
tf.random.set_seed(5)
sample_k_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=90, 
    top_k=50
)

print(tokenizer.decode(sample_k_output[0], skip_special_tokens=True))

Fargo is one of my favorite films of all time and in my opinion deserves to be in the top 10 films of the 90's. Even though the plot and story is one and the same story as the original, there are some parts that we can understand. The acting and the atmosphere of the film are beautiful and each of them make me feel like I am in such a dream life for the first time ever.The first hour, and two


In [79]:
tf.random.set_seed(5)
sample_k_output2 = model.generate(
    input_ids2, 
    do_sample=True, 
    max_length=90, 
    top_k=50
)

print(tokenizer.decode(sample_k_output2[0], skip_special_tokens=True))

Heath Ledger as the Joker really did a great job on the role, no denying that. Heath Ledger and Christopher Nolan both fit this role much as they did last night, though their performances aren't nearly as powerful as last night.I personally think Ledger's Joker is a good actor, but I think I have to say that Ledger is even worse than his first Batman, and I mean worse. <spoiler> false 


In [ ]:
topk_list = []
for sent in test_sent_list[0:501]:
  if len(sent)<=90:
    input_id4 = tokenizer.encode(sent, return_tensors='tf')
    tf.random.set_seed(6)
    topk_output = model.generate(input_id4, do_sample=True, max_length=90, top_k=50)
    topk_list.append(tokenizer.decode(topk_output[0], skip_special_tokens=True))
len_list = len(topk_list)
topk_spoil_alert_counter = 0
for elem in topk_list:
  if "<spoiler>" in elem:
    topk_spoil_alert_counter+=1

result_topk = topk_spoil_alert_counter/500
print(result_topk)

немного увеличим k:

In [80]:
tf.random.set_seed(5)
sample_k_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=90, 
    top_k=60
)

print(tokenizer.decode(sample_k_output[0], skip_special_tokens=True))

Fargo is one of my favorite films of all time and in my opinion deserves to be in the top 10 films of the 90's. Even though the plot and story is one and the same story as the original, there are some parts that we can understand. The acting and the atmosphere of the film are beautiful and each of them make me feel like I am in such a dream life for the first time ever.The first hour, and two


In [81]:
tf.random.set_seed(5)
sample_k_output2 = model.generate(
    input_ids2, 
    do_sample=True, 
    max_length=90, 
    top_k=60
)

print(tokenizer.decode(sample_k_output2[0], skip_special_tokens=True))

Heath Ledger as the Joker really did a great job on the role, no denying that. Heath Ledger and Christopher Nolan both fit this role much as they did last night, though their performances aren't nearly as powerful as last night.I personally think Ledger's Joker is a good actor, but I think I have to say that Ledger is even worse than his first Batman, and I mean worse. <spoiler> false 


уменьшим k:

In [82]:
tf.random.set_seed(5)
sample_k_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=90, 
    top_k=30
)

print(tokenizer.decode(sample_k_output[0], skip_special_tokens=True))

Fargo is one of my favorite films of all time and in my opinion deserves to be in the top 10 films of the 90's. Although I didn't like The Usual Suspects for much of the movie I had to give it a chance. However this is a great film, if not a masterpiece for many reasons because of the fact that it has great direction (I found myself in a tense scene on <spoiler> false  I


In [83]:
tf.random.set_seed(5)
sample_k_output2 = model.generate(
    input_ids2, 
    do_sample=True, 
    max_length=90, 
    top_k=30
)

print(tokenizer.decode(sample_k_output2[0], skip_special_tokens=True))

Heath Ledger as the Joker really did a great job on the role, no denying that. Heath Ledger and Christopher Nolan both had a chance to create something special, and with the support of Nolan, Ledger and Ledger proved they could not do it alone. It was a true accomplishment to see how much more he can do than Ledger. The entire time he was on screen and in the audience I was amazed how he could do


Top-p sampling

In [86]:
tf.random.set_seed(5)
sample_p_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=90, 
    top_p=0.92, 
    top_k=0
)

print(tokenizer.decode(sample_p_output[0], skip_special_tokens=True))

Fargo is one of my favorite films of all time and in my opinion deserves to be in the top 10 films of the 90's. Even though the plot and story is horribly nonsensical this film is as enjoyable to watch as it is controversial to read. You can watch this film over and over and over again for hours. And if you're a fan of any recent PIXAR movie then definitely watch this one. Again, I think most people


In [87]:
tf.random.set_seed(5)
sample_p_output2 = model.generate(
    input_ids2, 
    do_sample=True, 
    max_length=90, 
    top_p=0.92, 
    top_k=0
)

print(tokenizer.decode(sample_p_output2[0], skip_special_tokens=True))

Heath Ledger as the Joker really did a great job on the role, no denying that. Heath Ledger and Christopher Nolan both fit this role perfectly as the well-respected, though controversial Joker. The costume for him as well as his cape is a joy to behold. The Joker is so unconventional and creepy that I have never found myself shaking my head even once when he first approaches me and I can see it in his mannerisms - he


Повысим p:

In [89]:
tf.random.set_seed(5)
sample_p_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=90, 
    top_p=0.95, 
    top_k=0
)

print(tokenizer.decode(sample_p_output[0], skip_special_tokens=True))

Fargo is one of my favorite films of all time and in my opinion deserves to be in the top 10 films of the 90's. Even though the plot and story is horribly nonsensical this film is as enjoyable to watch as it is controversial to read. You can watch this film over and over and over again for hours. And if you're a fan of any recent PIXAR movie then definitely watch this one. Again, I think most people


In [90]:
tf.random.set_seed(5)
sample_p_output2 = model.generate(
    input_ids2, 
    do_sample=True, 
    max_length=90, 
    top_p=0.95, 
    top_k=0
)

print(tokenizer.decode(sample_p_output2[0], skip_special_tokens=True))

Heath Ledger as the Joker really did a great job on the role, no denying that. Heath Ledger and Christopher Nolan both fit this role perfectly as the well-respected, though controversial Joker we know and loathe him! Yes, the Batman troupe is hurt but because we all know the Joker we all love him because he's good and Batman even makes <spoiler> false  I had an outstanding preview from Indy on pix


In [ ]:
topp_list = []
for sent in test_sent_list[0:501]:
  if len(sent)<=90:
    input_id5 = tokenizer.encode(sent, return_tensors='tf')
    tf.random.set_seed(6)
    topp_output = model.generate(input_id5, max_length=90)
    topp_list.append(tokenizer.decode(topp_output[0], skip_special_tokens=True))
len_list = len(topp_list)
topp_spoil_alert_counter = 0
for elem in topp_list:
  if "<spoiler>" in elem:
    topp_spoil_alert_counter+=1

result_topp = topp_spoil_alert_counter/500
print(result_topp)